In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.optimize as sco

plt.style.use('seaborn-darkgrid')
sns.set(rc={'figure.figsize':(20,15)})
%matplotlib inline


In [2]:
%store -r LSTM_pred_IVV
%store -r LSTM_pred_EFA
%store -r LSTM_pred_EEM
%store -r LSTM_pred_VIG
%store -r LSTM_pred_SHY
%store -r LSTM_pred_LQD
%store -r LSTM_pred_EMB
%store -r LSTM_pred_MUB
%store -r LSTM_pred_TIP
%store -r LSTM_pred_VNQ
%store -r LSTM_pred_XLE

In [5]:
LSTM_pred = pd.concat([LSTM_pred_IVV,LSTM_pred_EFA,LSTM_pred_EEM,LSTM_pred_VIG,LSTM_pred_SHY,
                        LSTM_pred_LQD,LSTM_pred_EMB,
                       LSTM_pred_MUB,LSTM_pred_TIP,LSTM_pred_VNQ,LSTM_pred_XLE], axis=1)

In [10]:
LSTM_pred.columns = ['LSTM_pred_IVV','LSTM_pred_EFA','LSTM_pred_EEM','LSTM_pred_VIG'
                     ,'LSTM_pred_SHY',
                        'LSTM_pred_LQD','LSTM_pred_EMB',
                       'LSTM_pred_MUB','LSTM_pred_TIP','LSTM_pred_VNQ','LSTM_pred_XLE']
%store LSTM_pred
LSTM_pred.to_csv('LSTM_pred.csv')

Stored 'LSTM_pred' (DataFrame)


In [8]:
LSTM_pred

,LSTM_pred_IVV,LSTM_pred_EFA,LSTM_pred_EEM,LSTM_pred_VIG,LSTM_pred_SHY,LSTM_pred_LQD,LSTM_pred_EMB,LSTM_pred_MUB,LSTM_pred_TIP,LSTM_pred_VNQ,LSTM_pred_XLE
0,193.921326,57.398602,36.639183,73.598114,80.472343,100.548592,89.382248,97.721222,103.189110,64.188812,68.516998
1,194.122620,57.896080,36.751339,73.917885,80.498085,100.748283,89.204704,98.106560,103.287941,64.387321,67.910271
2,195.439819,57.765152,36.596054,74.548218,80.661179,101.323578,89.819191,98.431458,104.310608,65.487358,66.982140
3,193.903046,57.494598,36.509785,73.479324,80.555611,99.966583,89.592865,98.142662,103.261009,63.105137,67.524216
4,192.599350,56.901932,36.610092,73.148735,80.709190,100.413300,89.625191,98.160751,103.880409,63.915504,66.169983
5,192.783325,57.185829,35.446075,73.571388,80.707291,100.343742,89.894753,98.606453,103.602257,65.294075,65.286514
6,197.372833,58.082191,35.723881,74.499191,80.678436,100.696915,89.732498,98.850655,103.754807,65.920593,64.393547
7,193.206482,56.822094,34.250019,72.909843,80.659248,101.050018,89.164253,99.221405,103.853500,65.553040,61.841721
8,195.515274,57.611755,34.301929,74.058281,80.726494,101.463303,89.716248,99.311844,104.346451,66.329872,61.710041
9,193.142105,57.460896,33.593365,73.488686,80.582458,101.050018,88.750114,99.040543,103.835564,66.154472,59.609612


In [12]:
LSTM_pred_pct = LSTM_pred.pct_change(1).dropna()


In [13]:
returns_pred = LSTM_pred_pct
weighted_avg_ret = returns_pred.mean()
cov_matrix = returns_pred.cov()
runs = 50000
rf = 0.01

In [14]:
def portfolio_performance_annualized(weights, weighted_avg_ret, cov_matrix):
    returns_actual = np.sum(weighted_avg_ret*weights ) *52
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(52)
    return std, returns_actual

def random_portfolios(runs, weighted_avg_ret, cov_matrix, rf):
    results = np.zeros((3,runs))
    weights_record = []
    for i in range(runs):
        weights = np.random.random(11)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_performance_annualized(weights, weighted_avg_ret, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - rf) / portfolio_std_dev
    return results, weights_record

def simulated_pf_weights_only(weighted_avg_ret, cov_matrix, runs, rf):
    for i in range (10): 
        results, weights = random_portfolios(runs,weighted_avg_ret, cov_matrix, rf)

        max_sharpe_idx = np.argmax(results[2])
        sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
        max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=LSTM_pred.columns,columns=['allocation'])
        max_sharpe_allocation.allocation = [round(i,3)for i in max_sharpe_allocation.allocation]
        max_sharpe_allocation = max_sharpe_allocation.T
        
        min_vol_idx = np.argmin(results[0])
        sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
        
        print('-'*80)
        print('Optimal Sharpe Ratio Portfolio Allocation')
        print(max_sharpe_allocation)

In [15]:
simulated_pf_weights_only(weighted_avg_ret, cov_matrix, runs, rf)

--------------------------------------------------------------------------------
Optimal Sharpe Ratio Portfolio Allocation
            LSTM_pred_IVV  LSTM_pred_EFA  LSTM_pred_EEM  LSTM_pred_VIG  \
allocation          0.105          0.011          0.006          0.091   

            LSTM_pred_SHY  LSTM_pred_LQD  LSTM_pred_EMB  LSTM_pred_MUB  \
allocation          0.261           0.01          0.124          0.256   

            LSTM_pred_TIP  LSTM_pred_VNQ  LSTM_pred_XLE  
allocation          0.095          0.039          0.003  
--------------------------------------------------------------------------------
Optimal Sharpe Ratio Portfolio Allocation
            LSTM_pred_IVV  LSTM_pred_EFA  LSTM_pred_EEM  LSTM_pred_VIG  \
allocation          0.058          0.003          0.006          0.279   

            LSTM_pred_SHY  LSTM_pred_LQD  LSTM_pred_EMB  LSTM_pred_MUB  \
allocation          0.021          0.098          0.142          0.262   

            LSTM_pred_TIP  LSTM_pred_VNQ  